## Kubernetes on Azure

This notebook can be used to launch a Kubernetes Cluster on the [Azure](https://portal.azure.com). 

1. First make sure that [Azure CLI and kubectl are  installed on your computer](https://github.com/Azure/azure-cli).
2. Install kubectl. 
`az acs kubernetes install-cli`
2. Make sure [helm is installed](https://zero-to-jupyterhub-with-kubernetes.readthedocs.io/en/latest/setup-helm.html).


How big of a cluster do you need? Check out [this spreadsheet](https://docs.google.com/spreadsheets/d/1EvGMgS2JiGm8UuB9eDOQRjm79LapjJz4ubk7YDVqplY/edit?usp=sharing) or this 
[Notebook](https://github.com/data-8/jupyterhub-k8s/blob/master/docs/cost-estimation/gce_budgeting.ipynb) to estimate the size. 



## Configuration Setup

The general configuration is in `/config/azure`.



### Cluster Properties
*Resource Group (resource_group)*:  This will help with tracking costs and deleting things. 

*Locations* 

*Namesapece*: You can read more about namespaces [here](https://kubernetes.io/docs/concepts/overview/working-with-objects/namespaces/).

*Releasename*: A releasename is a way of versioning your system.  

*Zone*: See this advice from google on [choosing a zone](https://cloud.google.com/compute/docs/regions-zones/regions-zones#choosing_a_region_and_zone).

### Cluster Size and Autoscaling

*Number of Nodes (num_nodes)*: This is the number of servers which you are asking Google to launch.  This will clearly be one of the factors which drives the overall costs.

*Machine Type*: This is a (https://cloud.google.com/compute/docs/machine-types)

*Maximum Number of Nodes (max_nodes)*: If you enable autoscaling, Google will launch additional servers based on demand, up to the `maz Autoscaling is the process of increasing the number of servers based on demand.

**If you would like to make changes, just update the Kubernetes.yaml file.** 


In [ ]:
import sys 
import ruamel.yaml 
#This is your configuration file. 
general_yaml='../../config/config.yaml'
with open(general_yaml, 'r') as yaml:
    cf=ruamel.yaml.round_trip_load(yaml, preserve_quotes=True)

#This will import some required libraries.

azure_yaml='../../config/azure/config_tst.yaml'
#azure_yaml='../../config/azure/config_prd.yaml'
with open(azure_yaml, 'r') as a_yaml:
    cf_a=ruamel.yaml.round_trip_load(a_yaml, preserve_quotes=True)

if cf['docker']:
    cf_a['path']=cf['docker_path']
else:
    cf_a['path']=cf['local_path']

sys.path.append(cf_a['path']+"/lib/kuberutils") 
print(cf_a['path']+"/lib/kuberutils")
import importlib
import kuberutils as ku
importlib.reload(ku)
#This will load common commands for your cluster
cf_a=ku.azure_commands(cf_a)
print(ruamel.yaml.dump(cf_a, sys.stdout, Dumper=ruamel.yaml.RoundTripDumper))

### Web Login

In order to use the web login, you need to run the command below and then enter the code in the generated we link.  Careful not to commit this code to github repository. 

In [ ]:
#For some reason python function doesn't work.   
#Need Service 
!az login

### Service Account Login

TBD: Need someone to research loging in with service account. The way google works you can download a json file that can be used for authentication.  This is better for eventual automation. 


In [ ]:
#Login 
#ku.bash_command('login',cf_a) #tbd

### Create Resource Group and Location
Azure must have a resource group to assocate with your activities. This will make cleanup easier. We are going to call this a project to maintain consistent nameing with GC.

It is OK to run this more than once. 

In [ ]:
#This will set the create the project. 
result= ku.bash_command('create_project',cf_a)

#TBD, write a test to see if State= "Succeeded"
#'{\n  "id": "/subscriptions/652b3848-14d2-4276-af14-fbcd7db53805/resourceGroups/kuberlytics",\n  "location": "eastus",\n  "managedBy": null,\n  "name": "kuberlytics",\n  "properties": {\n    "provisioningState": "Succeeded"\n  },\n  "tags": null\n}\n'

### Create the Cluster
You are ready to go!  We are now going to tell Azure to create a Kubernetes cluster for us.  We do this with the `az acs create --orchestrator-type=kubernetes` 

See full documentation [here](https://docs.microsoft.com/en-us/cli/azure/acs?view=azure-cli-latest#az_acs_create).

#### Key factors
The number of nodes and the machine type are critica factors in the volume.


[Machine Sizes](https://docs.microsoft.com/en-us/azure/cloud-services/cloud-services-sizes-specs) 

[Machine Sizes](https://docs.microsoft.com/en-us/azure/virtual-machines/windows/sizes-memory)

Default is a Standard_D2_v2 which has 2 CPU and 7 GB Memory. 

For Google ~50 Students requires 2 4 CPU and 26 GB.

My thought is that 

Standard_E4s_v3 is the way to go for Production 
(2) While for Staging go with 1 

DS11 v2	2	14.00 GiB	28 GiB	~$137.64/mo
DS12 v2	4	28.00 GiB	56 GiB	~$276.03/mo

  "error": {
    "code": "InvalidParameter",
    "message": "The value 'Standard_E4s_v3' of parameter 'agentProfile.vmSize' is not allowed. Allowed values are 'Standard_A0, Standard_A1, Standard_A2, Standard_A3, Standard_A4, Standard_A5, Standard_A6, Standard_A7, Standard_A8, Standard_A9, Standard_A10, Standard_A11, Standard_D1, Standard_D2, Standard_D3, Standard_D4, Standard_D11, Standard_D12, Standard_D13, Standard_D14, Standard_D1_v2, Standard_D2_v2, Standard_D2_v2_Promo, Standard_D3_v2, Standard_D3_v2_Promo, Standard_D4_v2, Standard_D4_v2_Promo, Standard_D5_v2, Standard_D5_v2_Promo, Standard_D11_v2_Promo, Standard_DS11_v2, Standard_D12_v2, Standard_D12_v2_Promo, Standard_D13_v2, Standard_D13_v2_Promo, Standard_D14_v2, Standard_D14_v2_Promo, Standard_D15_v2, Standard_G1, Standard_G2, Standard_G3, Standard_G4, Standard_G5, Standard_A1_v2, Standard_A2_v2, Standard_A2m_v2, Standard_A4_v2, Standard_A4m_v2, Standard_A8_v2, Standard_A8m_v2, Standard_D15_v2, Standard_F1, Standard_F16, Standard_F2, Standard_F4, Standard_F8, Standard_H16, Standard_H16m, Standard_H16mr, Standard_H16r, Standard_H8, Standard_H8m, Standard_L16s, Standard_L32s, Standard_L4s, Standard_L8s, Standard_DS1, Standard_DS2, Standard_DS3, Standard_DS4, Standard_DS11, Standard_DS12, Standard_DS13, Standard_DS14, Standard_GS1, Standard_GS2, Standard_GS3, Standard_GS4, Standard_GS5, Standard_DS1_v2, Standard_DS2_v2, Standard_DS3_v2, Standard_DS4_v2, Standard_DS5_v2, Standard_DS11_v2, Standard_DS12_v2, Standard_DS13_v2, Standard_DS14_v2, Standard_DS15_v2, Standard_DS2_v2_Promo, Standard_DS3_v2_Promo, Standard_DS4_v2_Promo, Standard_DS5_v2_Promo, Standard_F16s, Standard_F1s, Standard_F2s, Standard_F4s, Standard_F8s, Standard_NC12, Standard_NC24, Standard_NC24r, Standard_NC6, Standard_NV12, Standard_NV24, Standard_NV6'


In [ ]:
#This will create the cluseter
#This should be run in a terminal because it is very long running. 
#However, it did work 
#print(cf_a['create_cluster'])
print(ku.bash_command('create_cluster_acs',cf_a))

#TBD WRITE TEST TO SEE IF "provisioningState": "Succeeded",

### Verifying the Servers
We can now verify that the cluster has successfully launched by asking gcloud to list the instances. This should report the number of instances specified in the num_nodes variable. 

`!gcloud compute instances list`


In [ ]:
servers=ku.bash_command('describe_cluster_acs',cf_a)
print(servers)
#TBD Need to parse this and just print out number of servers, size, not key. 

### Get Credentials for Kubectl
We need to add the credentials for Kubectl to work. 

In [ ]:
#gcloud container clusters get-credentials kuberlytics
print(ku.bash_command('get_credentials_acs',cf_a))


In [ ]:
#Check to see if we have Kubectl working. 
print(ku.bash_command("kubectl cluster-info"))


In [ ]:
print(ku.bash_command("kubectl get node"))



In [ ]:
print(ku.bash_command("kubectl get pods --all-namespaces"))

### Helm Installation.  
We are going to be utilizing Helm for  installations of a variety of analytics tools.  This command will install Tiller on your cluster.  As they say, "Happy Helming!" 

A critical factor for Helm is that you have the same version running locally and via your machine.  If you run helm version and you have the right version, then you should be fine.

To install the appropriate version, 

```
curl https://raw.githubusercontent.com/kubernetes/helm/master/scripts/get > get_helm.sh
chmod 700 get_helm.sh
RUN get_helm.sh --version v2.6.2

```


In [ ]:
print(ku.bash_command("helm version"))

In [ ]:
# Provided helm is installed, this will install Tiller on the Cluster.
#If the client and the server are not the same version, do an upgrade
#ACS already configured. 
print(ku.bash_command("helm init --client-only"))

### Enabling Autoscaling (optional)

This should launch a pod within your kubernetes cluster that will handle autoscaling of the cluster. Note that this seems to take a while and may even timeout. Consider opening and running in a terminal session. 

In [ ]:
#Currently Not possible with azure

### Fixed IP Address (optional)
Often it can be useful to utilize a fixed IP address in order to point a DNS to an applicaiton. 

[1] global
 [2] region: asia-east1
 [3] region: asia-northeast1
 [4] region: asia-southeast1
 [5] region: europe-west1
 [6] region: us-central1
 [7] region: us-east1
 [8] region: us-east4
 [9] region: us-west1

In [ ]:
#Reserve a fixed IP (note: you can only do this once.)
#print(ku.bash_command(cf_g['create_fixedip']))


In [ ]:
#fixed_ip=ku.get_fixed_ip(cf_g)
#print(fixed_ip)

In [ ]:
#Write the public IP to the Jupyterhub file.
#jupyterhub_yaml='../../config/jupyterhub/config.yaml'
#with open(jupyterhub_yaml, 'r') as j_yaml:
#    cf_j=ruamel.yaml.round_trip_load(j_yaml, preserve_quotes=True)
#cf_j['fixed_ip']=fixed_ip
#ruamel.yaml.round_trip_dump(cf_j, open(jupyterhub_yaml, 'w'))

### Kubernetes Web Dashboard
The kubernetes web dashboard can be utilized to launch applications. 

You the user for the dashboard is `admin` and the password can be found using the commands below. Just go ahead to the kubernetes dashbaord now. It is a great place to see the usage of your cluster and other things.

In [ ]:
#You can use this to show the Kubernetes Dashboard.
result=ku.bash_command("kubectl cluster-info")
print(result)
result=ku.bash_command('describe_cluster',cf_a)
#result=result.split("\n")
#password=[x for x in result if "password:" in x]
#print (password)

#TBD How do you get the Password for azure. 

In [ ]:
!kubectl cluster-info dump

#### That is it! You now have your own Kubernetes cluster that is ready to go. 

### Resize a Cluster
To stop a cluster without deleting it you just resize it to 0.

In [ ]:
#TBD
ku.bash_command(cf_a['class_size_cluster'])

In [ ]:
#TBD
#ku.bash_command(cf_g['stop_cluster'])

In [ ]:
#TBD
ku.bash_command(cf_a['normal_size_cluster'])

### Deleting a Kubernetes Cluster

# **WARNING: DELETE JUPYTERHUB INSTANCE FIRST 
Should really put that in the codebase.  If you don't delete the jupyterhub instance and you are using a fixed IP the forwarding rules for the fixed-ip won't be deleted.

This will delete the Kubernetes cluster.

In [ ]:
#Always delete the namespace first. 
print(ku.bash_command('delete_project',cf_a))

### Deleting Fixed IP Address

In [ ]:
#ku.bash_command('delete_fixedip',cf_g)

In [ ]:
print(ku.bash_command('describe_fixedip',cf_g))